In [1]:
import os
os.chdir('/Users/Bya/git/predictEPL/MyFunctions/')
import tokenizers
import senti_word_net
import featx
import replacers

from nltk.corpus import stopwords

import pickle
with open('/Users/Bya/nltk_data/taggers/treebank_aubt.pickle', 'rb') as f:
    tagger_aubt = pickle.load(f)

from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer

In [2]:
with open("/Users/Bya/git/predictEPL/NLTK/NLTK_TUTORIAL/pickled_algos/naiveBayes_for_short_reviews_NEW.pickle", "rb") as saved_classifier_f:
    nb_classifier = pickle.load(saved_classifier_f)

| WordNet tag        | Treebank tag| Word |
|-------------:| -----:|
| n      | NN | Noun |
| a      | JJ      |  Adjective | 
| r|RB | Adverb |
| v|VB | Verb |

In [3]:
tweets = [
"Great start to the evening seeing Mo Farah win the 5000m, now for #Arsenal. #Beijing2015 #NEWARS http://t.co/9ulMYqQ91M",
"Good squad there. Could be in with a very small chance #nufc",
"Blog: Newcastle United feedback: Will the Magpies replace like for like if they sell? http://t.co/rxfXZbGwVE #NUFC",
"FourFourTwo: Knee injury sidelines Ozil http://t.co/rxfXZbGwVE #NUFC",
"Blog: Newcastle United coach Steve Black has got in spot on once again http://t.co/rxfXZbGwVE #NUFC",
"RT @ghanafans: KICK OFF - #Arsenal will be attacking from right to left #NUFCvAFC  #BPL",
"I hate that andre marraner he is a total cock #nufc",
"Really dirty stuff from Sissoko, stomping on Nacho Monreal's ankle. Only a yellow but could have been a sending off. #Arsenal #NUFC",
"Really please to make my debut for this fantastic club. Great to get the win. Now we focus on Saturdays game #toonarmy #NUFC",
"RT @TheSunFootball: #Arsenal break well through Sanchez and Oxlade-Chamberlain. Terrible return pass from Ramsey wrecks the move. 0-0",
"RT @NUFC_Index: NUFC hunting in packs early on!",
"Gets a yellow for a stamp!#nufc",
"Come on gunners! #NUFCvAFC #Arsenal"]

In [4]:
def cleanHash(word):
    if word[0] == '#':
        return '#'
    elif word[0] == '@':
        return '@'
    elif word[0:4] == 'http':
        return 'http'
    else:
        return word

example = 1
for tweet in tweets:
    print("Example %d=====================================================\n" % example)
    example += 1

    print("Tweet: \n \n %s \n" % tweet)
    
    # can't -> cannot, bya's -> bya is
    replacer = replacers.RegexpReplacer()
    tweet = replacer.replace(tweet)
    
    # Tweet tokenizer
    words = TweetTokenizer().tokenize(tweet)
    print("Words(tweetTokenizer): \n\n %s \n" % words)
    
    # defining stopwords
    english_stops = set(stopwords.words('english'))
    english_stops_added = english_stops | {'.', ',', ':', ';', '#', '?', 'RT', '-', '@', 'rt'}
    words = [word.lower() for word in words if word.lower() not in english_stops_added]
    
    # Lemmatizer
    lemmatizer = WordNetLemmatizer()
    words = map(lambda word: lemmatizer.lemmatize(word), words)

    # words = map(lambda word: cleanHash(word), words)
    words = [cleanHash(word) for word in words]
    
    # defining stopwords
#     english_stops = set(stopwords.words('english'))
    english_stops_added =  {'.', ',', '!', ':', ';', '#', '?', 'RT', '-', '@', 'rt'}
    words = [word.lower() for word in words if word.lower() not in english_stops_added]
 
    print("Words(byaCleaner): \n \n %s \n" % words)

    print("\n---------------Naive Bayes--------------------------------------\n")
    
    probs = nb_classifier.prob_classify(featx.bag_of_words(words))
#     probs.samples()
    print("Naive Bayes Sentiment: %s" % probs.max())
    print("\t pos: %s" % probs.prob('pos'))
    print("\t neg: %s" % probs.prob('neg'))
    
    print("\n---------------SentiWordNet--------------------------------------\n")
    
    taggedAUBTwords = tagger_aubt.tag(words)
    
    print("POS tags: \n\n %s \n\n" % taggedAUBTwords)
    
    wordsWNtag = list(map(lambda word: senti_word_net.wordnet_sanitize(word), taggedAUBTwords))
    
    print("WordNet tags: \n\n %s \n\n" % wordsWNtag)
    
    sentiWords = [(word[0], senti_word_net.senti_word_net_word(word)) for word in wordsWNtag]
    
    print("SentiWordNet-Word: \n\n %s \n\n" % sentiWords)
    
    p,n,o = senti_word_net.pos_neg_score_words(wordsWNtag)
    
    if p > n:
        senti = 'pos'
    else:
        senti = 'neg'
    
    print("SentiWordNet-Tweet: %s \n \t\t pos: %s \n \t\t neg: %s \n \t\t obj: %s \n" % 
         (senti, p, n, o))

Example 1=====================================================

Tweet: 
 
 Great start to the evening seeing Mo Farah win the 5000m, now for #Arsenal. #Beijing2015 #NEWARS http://t.co/9ulMYqQ91M 

Words(tweetTokenizer): 

 ['Great', 'start', 'to', 'the', 'evening', 'seeing', 'Mo', 'Farah', 'win', 'the', '5000m', ',', 'now', 'for', '#Arsenal', '.', '#Beijing2015', '#NEWARS', 'http://t.co/9ulMYqQ91M'] 

Words(byaCleaner): 
 
 ['great', 'start', 'evening', 'seeing', 'mo', 'farah', 'win', '5000m', 'http'] 


---------------Naive Bayes--------------------------------------

Naive Bayes Sentiment: pos
	 pos: 0.8784748393924136
	 neg: 0.12152516060758803

---------------SentiWordNet--------------------------------------

POS tags: 

 [('great', 'JJ'), ('start', 'VB'), ('evening', 'NN'), ('seeing', 'VBG'), ('mo', 'NN'), ('farah', 'NNP'), ('win', 'VB'), ('5000m', 'NN'), ('http', 'NN')] 


WordNet tags: 

 [('great', 'a'), ('start', 'v'), ('evening', 'n'), ('seeing', 'v'), ('mo', 'n'), ('farah',